In [2]:
import pandas as pd 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import gensim
import gensim.corpora as corpora
import re
import numpy as np
from pprint import pprint
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)  
import spacy

In [3]:
papers = pd.read_csv('abstracts.csv')

In [4]:
papers.head()

,Publication Date,Title,Abstract,Relevance
0,2020-01-22,Functional assessment of cell entry and recept...,"Over the past 20 years, several coronaviruses ...",1.0
1,2020-02-02,Genome Detective Coronavirus Typing Tool for r...,"Summary Genome Detective is a web-based, user-...",1.0
2,2020-02-11,The effect of travel restrictions on the sprea...,Motivated by the rapid spread of a novel coron...,1.0
3,2020-03-23,Recombination and lineage-specific mutations l...,The recent outbreak of a new coronavirus (SARS...,1.0
4,2020-03-02,Evaluating the impact of international airline...,Global airline networks play a key role in the...,1.0


In [5]:
papers.shape

(82852, 4)

In [6]:
papers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82852 entries, 0 to 82851
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Publication Date  82852 non-null  object 
 1   Title             82851 non-null  object 
 2   Abstract          70571 non-null  object 
 3   Relevance         82852 non-null  float64
dtypes: float64(1), object(3)
memory usage: 2.5+ MB


In [7]:
papers1=papers.dropna(axis=0, subset=['Abstract'])

In [8]:
papers1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70571 entries, 0 to 82851
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Publication Date  70571 non-null  object 
 1   Title             70571 non-null  object 
 2   Abstract          70571 non-null  object 
 3   Relevance         70571 non-null  float64
dtypes: float64(1), object(3)
memory usage: 2.7+ MB


In [9]:
papers1[7323:7325]

,Publication Date,Title,Abstract,Relevance
7324,2020-06-06,A multi-group SEIRA model for the spread of CO...,The outbreak and propagation of COVID-19 have ...,0.72
7325,2020-06-06,"COVID-19 spreading in Rio de Janeiro, Brazil: ...",The recent Coronavirus has been spreading thro...,0.72


#### Data Preprocessing

In [10]:

data = papers1.Abstract.values.tolist()


data = [re.sub("[^A-Za-z0-9' ]+", ' ', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove  single quotes
data = [re.sub("\'", "", sent) for sent in data]


pprint(data[:1])

['Over the past 20 years several coronaviruses have crossed the species '
 'barrier into humans causing outbreaks of severe and often fatal respiratory '
 'illness Since SARS CoV was first identified in animal markets global '
 'viromics projects have discovered thousands of coronavirus sequences in '
 'diverse animals and geographic regions Unfortunately there are few tools '
 'available to functionally test these novel viruses for their ability to '
 'infect humans which has severely hampered efforts to predict the next '
 'zoonotic viral outbreak Here we developed an approach to rapidly screen '
 'lineage B betacoronaviruses such as SARS CoV and the recent 2019 nCoV for '
 'receptor usage and their ability to infect cell types from different species '
 'We show that host protease processing during viral entry is a significant '
 'barrier for several lineage B viruses and that bypassing this barrier allows '
 'several lineage B viruses to enter human cells through an unknown receptor

In [11]:
# Tokenization

#papers1['Abstract_Cleaned'] = papers1.apply(lambda row: (word_tokenize(row['Abstract_Cleaned'])), axis = 1)
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

data_words = list(sent_to_words(data))

print(data_words[:1])

[['over', 'the', 'past', 'years', 'several', 'coronaviruses', 'have', 'crossed', 'the', 'species', 'barrier', 'into', 'humans', 'causing', 'outbreaks', 'of', 'severe', 'and', 'often', 'fatal', 'respiratory', 'illness', 'since', 'sars', 'cov', 'was', 'first', 'identified', 'in', 'animal', 'markets', 'global', 'viromics', 'projects', 'have', 'discovered', 'thousands', 'of', 'coronavirus', 'sequences', 'in', 'diverse', 'animals', 'and', 'geographic', 'regions', 'unfortunately', 'there', 'are', 'few', 'tools', 'available', 'to', 'functionally', 'test', 'these', 'novel', 'viruses', 'for', 'their', 'ability', 'to', 'infect', 'humans', 'which', 'has', 'severely', 'hampered', 'efforts', 'to', 'predict', 'the', 'next', 'zoonotic', 'viral', 'outbreak', 'here', 'we', 'developed', 'an', 'approach', 'to', 'rapidly', 'screen', 'lineage', 'such', 'as', 'sars', 'cov', 'and', 'the', 'recent', 'ncov', 'for', 'receptor', 'usage', 'and', 'their', 'ability', 'to', 'infect', 'cell', 'types', 'from', 'differ

In [12]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['over', 'the', 'past', 'years', 'several', 'coronaviruses', 'have', 'crossed', 'the', 'species_barrier', 'into', 'humans', 'causing', 'outbreaks', 'of', 'severe', 'and', 'often', 'fatal', 'respiratory', 'illness', 'since', 'sars', 'cov', 'was', 'first', 'identified', 'in', 'animal', 'markets', 'global', 'viromics', 'projects', 'have', 'discovered', 'thousands', 'of', 'coronavirus', 'sequences', 'in', 'diverse', 'animals', 'and', 'geographic', 'regions', 'unfortunately', 'there', 'are', 'few', 'tools', 'available', 'to', 'functionally', 'test', 'these', 'novel', 'viruses', 'for', 'their', 'ability', 'to', 'infect_humans', 'which', 'has', 'severely', 'hampered', 'efforts', 'to', 'predict', 'the', 'next', 'zoonotic', 'viral', 'outbreak', 'here', 'we', 'developed', 'an', 'approach', 'to', 'rapidly', 'screen', 'lineage', 'such', 'as', 'sars', 'cov', 'and', 'the', 'recent', 'ncov', 'for', 'receptor', 'usage', 'and', 'their', 'ability', 'to', 'infect', 'cell', 'types', 'from', 'different', '

In [13]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

In [14]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [15]:
# Removing Stopwords
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

#stop_words = set(stopwords.words('english'))
#papers1['Abstract_Cleaned'] = papers1.apply(lambda row: ([w for w in row['Abstract_Cleaned'] if w not in stop_words]),axis=1)


In [16]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['past', 'year', 'several', 'coronaviruse', 'cross', 'specie', 'barrier', 'human', 'cause', 'outbreak', 'severe', 'often', 'fatal', 'respiratory', 'illness', 'first', 'identify', 'animal', 'market', 'global', 'viromic', 'project', 'discover', 'thousand', 'diverse', 'animal', 'geographic', 'region', 'unfortunately', 'tool', 'available', 'functionally', 'test', 'novel', 'virus', 'ability', 'severely', 'hamper', 'effort', 'predict', 'next', 'zoonotic', 'viral', 'outbreak', 'develop', 'approach', 'rapidly', 'screen', 'lineage', 'recent', 'receptor', 'usage', 'ability', 'infect', 'cell', 'type', 'different', 'specie', 'show', 'host', 'process', 'viral', 'entry', 'significant', 'barrier', 'several', 'lineage', 'virus', 'bypass', 'barrier', 'allow', 'several', 'lineage', 'virus', 'enter', 'human', 'cell', 'unknown', 'receptor', 'also', 'demonstrate', 'different', 'lineage', 'virus', 'recombine', 'gain', 'entry', 'human', 'cell', 'confirm', 'human', 'receptor', 'recently', 'emerge', 'ncov']]


In [37]:
#data_words_nostops = remove_stopwords(data_words)

In [17]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 2), (1, 1), (2, 1), (3, 2), (4, 1), (5, 1), (6, 3), (7, 1), (8, 1), (9, 3), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 2), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 2), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 4), (31, 1), (32, 1), (33, 1), (34, 4), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 2), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 3), (49, 1), (50, 1), (51, 1), (52, 1), (53, 3), (54, 1), (55, 1), (56, 1), (57, 1), (58, 2), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 2), (67, 1), (68, 4), (69, 1), (70, 1)]]


In [18]:
id2word[0]

'ability'

In [19]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('ability', 2),
  ('allow', 1),
  ('also', 1),
  ('animal', 2),
  ('approach', 1),
  ('available', 1),
  ('barrier', 3),
  ('bypass', 1),
  ('cause', 1),
  ('cell', 3),
  ('confirm', 1),
  ('coronaviruse', 1),
  ('cross', 1),
  ('demonstrate', 1),
  ('develop', 1),
  ('different', 2),
  ('discover', 1),
  ('diverse', 1),
  ('effort', 1),
  ('emerge', 1),
  ('enter', 1),
  ('entry', 2),
  ('fatal', 1),
  ('first', 1),
  ('functionally', 1),
  ('gain', 1),
  ('geographic', 1),
  ('global', 1),
  ('hamper', 1),
  ('host', 1),
  ('human', 4),
  ('identify', 1),
  ('illness', 1),
  ('infect', 1),
  ('lineage', 4),
  ('market', 1),
  ('ncov', 1),
  ('next', 1),
  ('novel', 1),
  ('often', 1),
  ('outbreak', 2),
  ('past', 1),
  ('predict', 1),
  ('process', 1),
  ('project', 1),
  ('rapidly', 1),
  ('recent', 1),
  ('recently', 1),
  ('receptor', 3),
  ('recombine', 1),
  ('region', 1),
  ('respiratory', 1),
  ('screen', 1),
  ('several', 3),
  ('severe', 1),
  ('severely', 1),
  ('show', 

#### Building LDA Topic Model

In [20]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [21]:

#pprint(lda_model.print_topics())
#doc_lda = lda_model[corpus]
# # Printing the Keywords in 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.067*"transmission" + 0.044*"mask" + 0.043*"contact" + 0.035*"person" + '
  '0.023*"isolation" + 0.020*"contact_trace" + 0.017*"wear" + 0.013*"scenario" '
  '+ 0.011*"consideration" + 0.011*"education"'),
 (1,
  '0.049*"patient" + 0.048*"covid" + 0.034*"study" + 0.024*"high" + '
  '0.023*"risk" + 0.018*"mortality" + 0.017*"age" + 0.016*"group" + '
  '0.015*"die" + 0.014*"year"'),
 (2,
  '0.035*"virus" + 0.025*"vaccine" + 0.023*"cell" + 0.018*"human" + '
  '0.014*"viral" + 0.013*"response" + 0.013*"protein" + 0.011*"immune" + '
  '0.009*"infection" + 0.009*"also"'),
 (3,
  '0.081*"care" + 0.052*"patient" + 0.029*"medical" + 0.029*"hospital" + '
  '0.015*"require" + 0.015*"facility" + 0.014*"team" + 0.014*"service" + '
  '0.013*"management" + 0.013*"recommendation"'),
 (4,
  '0.054*"health" + 0.020*"pandemic" + 0.019*"people" + 0.016*"social" + '
  '0.013*"public" + 0.012*"mental" + 0.012*"impact" + 0.011*"physical" + '
  '0.010*"work" + 0.010*"home"'),
 (5,
  '0.046*"patient" +

In [23]:
# Visualizing topics using pyLDAvis

# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.111580  0.039243       1        1  20.446201
7      0.118410  0.042458       2        1  19.995566
8      0.127984  0.035586       3        1  15.196650
1      0.066534  0.119080       4        1  13.364581
2      0.147205 -0.118760       5        1   7.566762
4     -0.089981  0.176748       6        1   6.885961
3     -0.268173  0.234719       7        1   5.587124
6      0.077950 -0.130450       8        1   5.265794
9      0.040911 -0.128287       9        1   3.189454
0     -0.332419 -0.270336      10        1   2.501909, topic_info=            Term           Freq          Total Category  logprob  loglift
500      patient  237639.000000  237639.000000  Default  30.0000  30.0000
736         care   62318.000000   62318.000000  Default  29.0000  29.0000
59          test   53980.000000   53980.000000  Default  28.0000  28.0000
460       health   68094.000000   68094.000000  Default  27.0000  27.0000
732    treatment   44660.000000   44660.000000  Default  26.0000  26.0000
...          ...            ...            ...      ...      ...      ...
68         virus    3107.502156   48018.053824  Topic10  -4.6166   0.9504
415    infection    3194.805963   75803.926180  Topic10  -4.5889   0.5215
4133   situation    1936.344486   10163.567099  Topic10  -5.0897   2.0301
33        infect    2015.903480   16522.857406  Topic10  -5.0494   1.5844
291   individual    2055.739990   23182.146813  Topic10  -5.0298   1.2654

[609 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
1829      2  0.999849    access
1158      3  0.041411  accuracy
1158      8  0.958328  accuracy
442       5  0.999727       ace
333       3  0.048563  activity
...     ...       ...       ...
853       8  0.041350    worker
69        3  0.037258      year
69        4  0.840768      year
69        6  0.121920      year
70        5  0.999407  zoonotic

[1109 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 8, 9, 2, 3, 5, 4, 7, 10, 1])

In [26]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # measuring the model accuracy.


Perplexity:  -7.974149709955088


In [ ]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)